In [1]:
#from veneer.manage import create_command_line

#import veneer-py

from veneer.manage import start, create_command_line, kill_all_now
import veneer
import geopandas
import pandas as pd

In [2]:
veneer_install = 'C:/PythonScriptingForGBR/VeneerPlugin/v411'
source_version = '4.1.1'
cmd_directory = 'C:\\tempveneer\\veneer_cmd'
path = create_command_line(veneer_install,source_version,dest=cmd_directory)
path

'C:\\tempveneer\\veneer_cmd\\FlowMatters.Source.VeneerCmd.exe'

In [5]:
catchment_project='C:\\Beckers\\Beckers_GBR_411_Seasonal.rsproj'
num_copies=1    # Important - set this to be a number ~ the number of CPU cores in your system!
first_port=9950

In [6]:
processes,ports = start(catchment_project,
                        n_instances=num_copies,
                        ports=first_port,
                        debug=True,
                        veneer_exe=path,
                        remote=False)

Starting C:\tempveneer\veneer_cmd\FlowMatters.Source.VeneerCmd.exe -p 9950 -s  "C:\Beckers\Beckers_GBR_411_Seasonal.rsproj"
[0] Loading plugins

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\DERMTools.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\Dynamic_SedNet.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\GBR_DynSed_Extension.dll

[0] Plugins loaded (3/3)

[0] Opening project file: C:\Beckers\Beckers_GBR_411_Seasonal.rsproj

ERROR[0] log4net:ERROR Failed to find configuration section 'log4net' in the application's .config file. Check your .config file for the <log4net> and <configSections> elements. The configuration section should look like: <section name="log4net" type="log4net.Config.Log4NetConfigurationSectionHandler,log4net" />

[0] Loading project

[0] Project Loaded

[0] New Project

[0] Scenario 1

[0] [2:27:40 PM] Veneer, by Flow Matters: http://www.flowmatters.com.au

[0] [2:27:40 PM] Started Source RESTful Service on po

In [7]:
vs = [veneer.Veneer(port=p) for p in ports]

In [8]:
#kill_all_now(processes)

In [9]:
%matplotlib inline
v = vs[0]
#v.network().as_dataframe().plot()

In [10]:
sDate = v.model.get('scenario.CurrentConfiguration.StartDate')
print(sDate)

eDate = v.model.get('scenario.CurrentConfiguration.EndDate')
print(eDate)

#v.run_model()

1/01/2000 12:00:00 AM
31/12/2006 12:00:00 AM


In [11]:
the_network = v.network()
outlets = the_network.outlet_nodes()
outlets

[{'type': 'Feature', 'properties': {'name': 'Outlet Node1', 'icon': '/resources/ConfluenceNodeModel', 'feature_type': 'node'}, 'id': '/network/nodes/1', 'geometry': {'type': 'Point', 'coordinates': [388581.15296203, -2647562.2490278]}}]

In [12]:
def processLinks(theNetwork, allLinkAreas, currentLink, knownDownStreamLinksToUpdate):
    
    theLinkName = currentLink['properties']['name']
    theLinkID = currentLink['id']
    
    # I think this will actually return a catchment record, not just an ID
    theCat = theNetwork['features'].find_by_link(theLinkID)[0]
    
    #print(theCat)
    
    thisCatName = theCat['properties']['name']
    thisCatArea = theCat['properties']['areaInSquareMeters']
    
    #print("Processing " + thisCatName + " with area of " + str(thisCatArea))
    
    allLinkAreas[theLinkName] += thisCatArea
    
    newDSLinksToUpdate = []
    for eachDSLink in knownDownStreamLinksToUpdate:
        lName = str(eachDSLink)
        #print("DS Link " + lName + " with area of ")
        allLinkAreas[lName] += thisCatArea
        newDSLinksToUpdate.append(lName)
    
    # now add the link we've just processed
    newDSLinksToUpdate.append(theLinkName)
    
    #print("Link info: " + str(currentLink))
    
    fNodeID = currentLink['properties']['from_node']
    
    #print("Node info: " + fNodeID)
    
    fNode = the_network['features'].find_by_id(fNodeID)
    #print(fNode)
    
    fNodeName = fNode[0]['properties']['name']
    #print(fNodeName)
    
    ## recurse
    #print(the_network.upstream_links(fNode[0]))
    for upLink in the_network.upstream_links(fNode[0]):
        #tNodeID = upLink['properties']['to_node']
        #tNode = the_network['features'].find_by_id(tNodeID)
        #print("Now processing: " + upLink['properties']['name'] + " upstream of " + tNode[0]['properties']['name'] + " with initial Region: " + currentReg)
        processLinks(theNetwork, allLinkAreas, upLink, newDSLinksToUpdate)

    return

In [13]:
noRegStr = 'NoReg'
gaugeList = ['Outlet Node1']
#gaugeList = ['120205A','120206A','120207A','120209A','120212A','120213A','120214A','120219A']

# will assign regions to links, pair with subcatchs late
all_links = the_network['features'].find_by_feature_type('link')
linkAreas = {}

## Populate the dictionary of relationships first
for link in all_links:
    linkAreas[link['properties']['name']] = 0

## now process the network
for outNode in outlets:
    curReg = noRegStr
    
    #print(outNode.keys())
    #print(outNode['properties'].keys())
    #print(outNode['properties']['name'])
    
    thisNodeName = outNode['properties']['name']
    
    #this will be an empty list, built up for each link as we get to it
    downStreamLinksToUpdate = []
    
    ## now we begin processing
    for upLink in the_network.upstream_links(outNode):
        processLinks(the_network, linkAreas, upLink, downStreamLinksToUpdate)
    

In [14]:
linkAreas_df = pd.DataFrame(list(linkAreas.items()), columns=['LinkName','AreaM2'])
linkAreas_df
#help(v.network)
#help(run_results2)

,LinkName,AreaM2
0,link for catchment SC #3,14516406250
1,link for catchment SC #2,27716406250
2,link for catchment SC #4,5306250000
3,link for catchment SC #1,38858984375
4,link for catchment SC #5,8604687500
5,Link #1,0


In [15]:
network_df = v.network().as_dataframe()
catchdf = network_df[network_df.feature_type=='catchment']
linkdf = network_df[network_df.feature_type=='link']

catchlinkdf = pd.merge(catchdf, linkdf, how='inner', left_on=['link'], right_on=['id'])
catchlinkdf = catchlinkdf.rename(columns={'areaInSquareMeters_y':'linkarea', 'areaInSquareMeters_x':'areaInSquareMeters', 'feature_type_y':'link_feat_type', 'feature_type_x':'feature_type', 'geometry_y':'link_geom', 'geometry_x':'geometry', 'name_y': 'link_name'})

catchlinkdfregdf = pd.merge(catchlinkdf, linkAreas_df, how='inner', left_on=['link_name'], right_on=['LinkName'])
catchlinkdfregdf = catchlinkdfregdf.rename(columns={'name_x':'SubCat'})
catchlinkdfregdf.head(8)

,areaInSquareMeters,feature_type,from_node_x,geometry,icon_x,link_x,SubCat,to_node_x,id_x,linkarea,link_feat_type,from_node_y,link_geom,icon_y,link_y,link_name,to_node_y,id_y,LinkName,AreaM2
0,1.451641e+10,catchment,NaN,"(POLYGON ((300143.65296203 -2787874.7490278, 3...",NaN,/network/link/2,SC #3,NaN,/network/catchments/17,NaN,link,/network/nodes/9,"LINESTRING (344831.15296203 -2831312.2490278, ...",NaN,NaN,link for catchment SC #3,/network/nodes/6,/network/link/2,link for catchment SC #3,14516406250
1,7.893750e+09,catchment,NaN,"(POLYGON ((386393.65296203 -2690374.7490278, 3...",NaN,/network/link/1,SC #2,NaN,/network/catchments/26,NaN,link,/network/nodes/6,"LINESTRING (387331.15296203 -2811312.2490278, ...",NaN,NaN,link for catchment SC #2,/network/nodes/3,/network/link/1,link for catchment SC #2,27716406250
2,8.604688e+09,catchment,NaN,"(POLYGON ((309518.65296203 -2607874.7490278, 3...",NaN,/network/link/4,SC #5,NaN,/network/catchments/35,NaN,link,/network/nodes/11,"LINESTRING (376081.15296203 -2690687.2490278, ...",NaN,NaN,link for catchment SC #5,/network/nodes/3,/network/link/4,link for catchment SC #5,8604687500
3,5.306250e+09,catchment,NaN,"(POLYGON ((350143.65296203 -2735374.7490278, 3...",NaN,/network/link/3,SC #4,NaN,/network/catchments/44,NaN,link,/network/nodes/10,"LINESTRING (379206.15296203 -2803187.2490278, ...",NaN,NaN,link for catchment SC #4,/network/nodes/6,/network/link/3,link for catchment SC #4,5306250000
4,2.537891e+09,catchment,NaN,"(POLYGON ((390768.65296203 -2645374.7490278, 3...",NaN,/network/link/0,SC #1,NaN,/network/catchments/53,NaN,link,/network/nodes/3,"LINESTRING (384831.15296203 -2690687.2490278, ...",NaN,NaN,link for catchment SC #1,/network/nodes/1,/network/link/0,link for catchment SC #1,38858984375


In [16]:
catchlinkdfregdf.to_csv('C:/Beckers/BecksContribAreas.csv', columns=['SubCat', 'LinkName', 'AreaM2'], index=False)

In [17]:
kill_all_now(processes)